# Text Normalization (Chinese)

- `text_normalizer_zh.py`
- Including functions for:
    - word-seg chinese texts
    - clean up texts by removing duplicate spaces and line breaks
    - remove incompatible weird characters

In [1]:
import unicodedata
import re
#from nltk.corpus import wordnet
#import collections
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
import requests
import pandas as pd
import text_normalizer_zh as tnz

## Normalization Functions

In [2]:
# %load text_normalizer_zh.py
"""

Notes
-----

    These functions are based on the text normalization functions 
    provided in Text Analytics with Python 2ed.

"""

import unicodedata
import re
# from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import jieba

## Initialize Trad Chinese dictionary
jieba.set_dictionary('../../../RepositoryData/data/jiaba/dict.txt.jiebatw.txt')


## Normalize unicode characters
def remove_weird_chars(text):
    #     ```
    #     (NFKD) will apply the compatibility decomposition, i.e.
    #     replace all compatibility characters with their equivalents.
    #     ```
    text = unicodedata.normalize('NFKD', text).encode('utf-8',
                                                      'ignore').decode(
                                                          'utf-8', 'ignore')
    return text


## Remove extra linebreaks
def remove_extra_linebreaks(text):
    lines = text.split(r'\n+')
    return '\n'.join(
        [re.sub(r'[\s]+', ' ', l).strip() for l in lines if len(l) != 0])


## Remove extra medial/trailing/leading spaces
def remove_extra_spaces(text):
    return re.sub("\\s+", " ", text).strip()


## Seg the text into words
def seg(text):
    text_seg = jieba.cut(text)
    out = ' '.join(text_seg)
    return out


## Remove punctuation/symbols
def remove_symbols(text):
    """
    
    Unicode 6.0 has 7 character categories, and each category has subcategories:

    Letter (L): lowercase (Ll), modifier (Lm), titlecase (Lt), uppercase (Lu), other (Lo)
    Mark (M): spacing combining (Mc), enclosing (Me), non-spacing (Mn)
    Number (N): decimal digit (Nd), letter (Nl), other (No)
    Punctuation (P): connector (Pc), dash (Pd), initial quote (Pi), final quote (Pf), open (Ps), close (Pe), other (Po)
    Symbol (S): currency (Sc), modifier (Sk), math (Sm), other (So)
    Separator (Z): line (Zl), paragraph (Zp), space (Zs)
    Other (C): control (Cc), format (Cf), not assigned (Cn), private use (Co), surrogate (Cs)
    
    
    There are 3 ranges reserved for private use (Co subcategory): 
    U+E000—U+F8FF (6,400 code points), U+F0000—U+FFFFD (65,534) and U+100000—U+10FFFD (65,534). 
    Surrogates (Cs subcategory) use the range U+D800—U+DFFF (2,048 code points).
    
    
    """

    ## Brute-force version: list all possible unicode ranges, but this list is not complete.
    #   text = re.sub('[\u0021-\u002f\u003a-\u0040\u005b-\u0060\u007b-\u007e\u00a1-\u00bf\u2000-\u206f\u2013-\u204a\u20a0-\u20bf\u2100-\u214f\u2150-\u218b\u2190-\u21ff\u2200-\u22ff\u2300-\u23ff\u2460-\u24ff\u2500-\u257f\u2580-\u259f\u25a0-\u25ff\u2600-\u26ff\u2e00-\u2e7f\u3000-\u303f\ufe50-\ufe6f\ufe30-\ufe4f\ufe10-\ufe1f\uff00-\uffef─◆╱]+','',text)

    text = ''.join(ch for ch in text
                   if unicodedata.category(ch)[0] not in ['P', 'S'])
    return text


## Remove numbers
def remove_numbers(text):
    return re.sub('\\d+', "", text)


## Remove alphabets
def remove_alphabets(text):
    return re.sub('[a-zA-Z]+', '', text)


## Combine every step
def normalize_corpus(corpus,
                     is_remove_extra_linebreaks=True,
                     is_remove_weird_chars=True,
                     is_seg=True,
                     is_remove_symbols=True,
                     is_remove_numbers=True,
                     is_remove_alphabets=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        if is_remove_extra_linebreaks:
            doc = remove_extra_linebreaks(doc)

        if is_remove_weird_chars:
            doc = remove_weird_chars(doc)

        if is_seg:
            doc = seg(doc)

        if is_remove_symbols:
            doc = remove_symbols(doc)

        if is_remove_alphabets:
            doc = remove_alphabets(doc)

        if is_remove_numbers:
            doc = remove_numbers(doc)

        normalized_corpus.append(remove_extra_spaces(doc))

    return normalized_corpus

## Extract an Article

- Grab the first article from Google news for demonstration

In [3]:
url = 'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNRFptTXpJU0JYcG9MVlJYS0FBUAE?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
r = requests.get(url)
web_content = r.text
soup = BeautifulSoup(web_content, 'lxml')
title = soup.find_all('a', class_='DY5T1d')
first_art_link = title[0]['href'].replace('.', 'https://news.google.com', 1)

#print(first_art_link)
art_request = requests.get(first_art_link)
art_request.encoding = 'utf8'
soup_art = BeautifulSoup(art_request.text, 'lxml')

art_content = soup_art.find_all('p')
art_texts = [p.text for p in art_content]
print(art_texts)

['被網友封為「村長」的國民黨桃園市議員詹江村，昨被周刊爆料帶女粉絲上汽車旅館硬上，女子控訴詹射後不理；對此，詹江村昨天到桃園市警局桃園警分局對爆料女子提告妨害名譽；爆料女子今天也到新北市永和警分局提告詹涉嫌妨害性自主，警方已依規定受理，近期也將通知詹江村到案說明。', '\n', '\r\n綽號「村長」的桃園市議員詹江村昨被周刊爆料，去年8月帶女粉絲上汽車旅館硬上得逞，事後詹「射後不理」讓女生心寒，詹江村昨天否認女粉絲指控，同時也向桃園警方提告爆料女子妨害名譽。', '\n', '\r\n爆料女子認為詹敢做不敢承認，今天下午也檢具相關證據，前往新北市永和警分局報案，對詹江村提告涉嫌妨害性自主。警方已依規定受理，為釐清相關案情，不排除近期內通知詹江村到案說明。桃園市議員詹江村。圖／報系資料照', '\n                    新竹縣新豐鄉一棟磚造平房昨天凌晨發生火警，廿五歲姜姓女子因為起火點就在隔房雜物間，大火困住房門，加上鐵窗無法逃生，雖在窗...                  ', '\n                    四十一歲葉姓男子昨天中午駕駛滿載廢土的聯結車，行經國道一號路竹北上路段突擦撞前方小客車後翻覆，車體和滿車砂土正好壓中另一...                  ', '\n                    9月底，在高雄經營水產行的蔡姓男子，駕貨車在88線快速道路，超車側撞騎大型重機鄭姓男子，鄭摔倒地身上多處擦傷，影像曝光引發眾怒，許多人大罵根本是謀殺。交通部長林佳龍把這種惡意逼車者，視為全民公敵，瑞芳分局去年起在台二線抓逼車，多次目睹逼車實況，邊蒐證邊冷汗直流。                  ', '\n                    台北市大安區22日傳出瓦斯外洩，晚間消防人員會同瓦斯公司尋找洩漏點，目前已經針對可能的瓦斯洩漏範圍警戒。                  ', '\n                    濁水溪南岸今天下午再傳火警，雲林縣消防局已派員前往滅火，濃煙隨著東北季風籠罩雲林縣崙背鄉，空氣品質亮橘燈，雲林縣衛生局提...                  ', '\n                    27歲林姓男子稍早駕駛一輛保時捷行徑新北市淡水區淡金路時，因不明

- Normalized results:

In [4]:
tnz.normalize_corpus([' '.join(art_texts)])

Building prefix dict from /Users/Alvin/GoogleDrive/_MySyncDrive/RepositoryData/data/jiaba/dict.txt.jiebatw.txt ...


Loading model from cache /var/folders/n7/ltpzwx813c599nfxfb94s_640000gn/T/jieba.u1b52b47246a0f2e6497af6bbe107adac.cache


Loading model cost 0.563 seconds.


Prefix dict has been built successfully.


['被 網友 封為 村長 的 國民黨 桃園市 議員 詹江村 昨被 周刊 爆料 帶女 粉絲 上 汽車 旅館 硬上 女子 控訴 詹射 後 不理 對此 詹江村 昨天 到 桃園市 警局 桃園 警分局 對 爆料 女子 提告 妨害 名譽 爆料 女子 今天 也 到 新 北市 永和 警分局 提告 詹 涉嫌 妨害性 自主 警方 已依 規定 受理 近期 也將 通知 詹江村 到案 說明 綽號 村長 的 桃園市 議員 詹江村 昨被 周刊 爆料 去年 月 帶 女 粉絲 上 汽車 旅館 硬上 得逞 事 後 詹 射 後 不理 讓 女生 心寒 詹江村 昨天 否認 女 粉絲 指控 同時 也 向 桃園 警方 提告 爆料 女子 妨害 名譽 爆料 女子 認為 詹敢 做 不敢 承認 今天 下午 也 檢具 相關 證據 前往 新 北市 永和 警分局 報案 對 詹江村 提告 涉嫌 妨害性 自主 警方 已依 規定 受理 為 釐清 相關 案情 不 排除 近期 內 通知 詹江村 到案 說明 桃園市 議員 詹江村 圖 報系 資料 照 新竹縣 新豐鄉 一棟 磚造 平房 昨天 凌晨 發生 火警 廿五歲 姜姓 女子 因為 起火點 就 在 隔房 雜物 間 大火 困住 房門 加上 鐵窗 無法 逃生 雖 在 窗 四十一歲 葉姓 男子 昨天 中午 駕駛 滿載 廢土 的 聯結車 行經 國道 一號 路竹 北上 路段 突 擦撞 前方 小 客車 後 翻覆 車體 和 滿車 砂土 正好 壓中 另 一 月底 在 高雄 經營 水產 行 的 蔡姓 男子 駕 貨車 在 線 快速 道路 超車 側撞 騎 大型 重機 鄭姓 男子 鄭 摔倒 地身 上 多處 擦傷 影像 曝光 引發 眾怒 許多 人 大罵 根本 是 謀殺 交通部長 林佳龍 把 這種 惡意 逼車 者 視為 全民 公敵 瑞芳 分局 去年 起 在 台二 線 抓 逼車 多次 目睹 逼車 實況 邊 蒐證 邊 冷汗 直流 台北市 大安區 日 傳出 瓦斯 外洩 晚間 消防 人員 會同 瓦斯 公司 尋找 洩漏 點 目前 已經 針對 可能 的 瓦斯 洩漏 範圍 警戒 濁水溪 南岸 今天 下午 再傳 火警 雲林縣 消防局 已 派員 前往 滅火 濃煙 隨著 東北 季風 籠罩 雲林縣 崙背鄉 空氣 品質 亮橘 燈 雲林縣 衛生局 提 歲 林姓 男子 稍早 駕駛 一輛 保時捷 行徑 新 北市 淡水區 淡金路 時 因 不明 原